# ase-toolbox.ipynb
ASEによる化学シミュレーションでよく使うコードと、作ったヘルパー関数をまとめておくやつ。

### 🟡計算機を用意する(Matlantis環境)

In [ ]:
import pfp_api_client
from pfp_api_client.pfp.calculators.ase_calculator import ASECalculator
from pfp_api_client.pfp.estimator import Estimator, EstimatorCalcMode

# バージョンによって計算結果が異なる場合があるため、毎回バージョンを確認する
print(f"pfp_api_client: {pfp_api_client.__version__}")

# ASECalculatorを使用するための設定
# EstimatorCalcModeは、以下のように使い分ける
# - 一般の系： EstimatorCalcMode.CRYSTAL_U0 Uなしモード
# - 酸化物など： EstimatorCalcMode.CRYSTAL　Uありモード
# - 単体有機分子： EstimatorCalcMode.MOLECULE 分子モード
estimator = Estimator(calc_mode=EstimatorCalcMode.MOLECULE)
calculator = ASECalculator(estimator)  # このcalculatorをatoms.calcに設定して使用する


### 🟡matplotlibで日本語を表示する

In [ ]:
# ---matplotlibで日本語を表示する
plt.rcParams['font.family'] = ['DejaVu Sans', 'Hiragino Sans', 'Yu Gothic', 'Meiryo', 'Takao', 'IPAexGothic', 'IPAPGothic', 'VL PGothic', 'Noto Sans CJK JP']
# Windows環境での代替設定
try:
    import matplotlib.font_manager as fm
    # 日本語フォントを探す
    font_list = [f.name for f in fm.fontManager.ttflist if 'gothic' in f.name.lower() or 'mincho' in f.name.lower() or 'meiryo' in f.name.lower()]
    if font_list:
        plt.rcParams['font.family'] = font_list[0]
    else:
        # フォールバック: Unicode対応
        plt.rcParams['font.family'] = 'DejaVu Sans'
except:
    pass

plt.rcParams['font.family']

### 🟡可視化する

In [ ]:
from pfcc_extras.visualize.view import view_ngl
from ase.build import bulk

view_ngl(bulk("Cu"), representations=["ball+stick"], w=400, h=300)

### 🟡最適化

#### 最適化アルゴリズムについて
- 局所最適へのたどり着き方(=アルゴリズム)は、いろいろある
- 選び方
    - 基本的に`LBFGSLineSearch`か`FIRE`を使えば良いらしい
        - `LBFGSLineSearch`がうまくいかなかったら`FIRE`を使う、みたいな
    - 選ぶのが面倒なら、上の2つを組み合わせた`FIRELBFGS`を使うと良い
        - Matlantisオリジナルの最適化アルゴリズム
        - 使用方法は、2つ下のセルに示す
    - 局所最適を探すアルゴリズムの違いなだけなので、シミュレーションに大きく差が生じることはないのかもしれない
- 参考文献
    - https://docs.matlantis.com/atomistic-simulation-tutorial/ja/2_3_opt-algorithm.html

In [ ]:
from ase.optimize import FIRE, LBFGSLineSearch

opt = FIRE(atoms)
opt.run(fmax=0.001)


In [ ]:
# FIRELBFGS 最適化アルゴリズムの使い方(Matlantisオリジナル)
from matlantis_features.ase_ext.optimize import FIRELBFGS

opt = FIRELBFGS(atoms)
opt.run(fmax=0.001)


### 🟡書き出す

In [ ]:
# 内蔵ライブラリからcifファイルを作成する
from ase.build import molecule
from ase.io import write

# 出力先ディレクトリ（必要に応じて変更）
output_dir = "./"

# 分子リスト
species = ["CO2", "CO", "H2", "H2O", "O2"]

for sym in species:
    mol = molecule(sym)  # ASE 内蔵の分子ライブラリから生成
    filename = f"{output_dir}{sym}.cif"
    write(filename, mol)  # CIF 形式で保存
    print(f"Written: {filename}")


## 🔴構造を作る

### 🟡基本的な構造

In [ ]:
from ase.build import molecule, bulk, surface
from ase.io import read, write

# ---分子
co = molecule("CO")

# ---結晶
cu = bulk("Cu")
# 増やす
_cu = cu * (2, 3, 4)

# ---表面
slab = surface(
    lattice=cu, indices=(1, 1, 1), layers=2, vacuum=10.0
)  # cuの、(111)面を2層で真空層10.0Åで切り出す

# ---ファイルから
# cif,xyz,vaspからいけるらしい
# cu2o=read('Cu2O.cif')

# ---構造のコピー
# Pythonのルールとして、cu_copy = cu とするだけだと、cu_copyもcuも同じものを指し示してしまう
# (「Python 参照渡し」とかを調べるとわかる)
# copy()を使えば、別のオブジェクトとして扱える


cu_copy = cu.copy()
cu_copy[0].symbol = "O"  # コピー先の原子を変えてみる
print("=== cu_copy = cu.copy()の場合 ===")
print(f"コピー元: {cu[0].symbol}")  # コピー元: Cu
print(f"コピー先: {cu_copy[0].symbol}")  # コピー先: O
# copy()することで、コピー元とコピー先が別のものを指し示すようになる

cu_ref = cu
cu_ref[0].symbol = "O"
print("=== cu_ref = cuの場合 ===")
print(f"コピー元: {cu_ref[0].symbol}")  # コピー元: O
print(f"コピー先: {cu[0].symbol}")  # コピー先: O
# そのまま = で代入すると、コピー元とコピー先が同じものを指し示してしまうので、どちらも置き換わってしまう


### 🟡クラスター

In [ ]:
from ase.cluster import Icosahedron, Octahedron
from pfcc_extras.visualize.view import view_ngl

# ---八面体
# 正八面体
_cluster = Octahedron(
    "Cu",
    length=7,  # 層の数
    cutoff=0,  # 頂点をどれくらい切るか。0だと切らない。
)

# 切頂八面体: 頂点が切られている八面体
_cluster = Octahedron("Cu", length=7, cutoff=2)

# 正切頂八面体
_cluster = Octahedron(
    "Cu",
    length=7,  # length=3*cutoff+1で、正切頂八面体となる
    cutoff=2,
)

# 立方八面体
_cluster = Octahedron(
    "Cu",
    length=5,  # length=2*cutoff+1で、立方八面体となる
    cutoff=2,
)

# ---二十面体
_cluster = Icosahedron(
    "Cu",
    noshells=5,  # 原子の数
)

# クラスターの原子数の確認
print(f"原子数: {len(_cluster)}")

# 構造を可視化してみる
view_ngl(_cluster, representations=["ball+stick"], w=400, h=300)


### 🟡特定の原子を探す

In [ ]:
from FindAtoms import *
from ase.cluster import Icosahedron
from ase.build import surface, bulk

test_slab = surface(bulk("Cu"), indices=(1, 1, 1), layers=2, vacuum=10.0)
test_cluster = Icosahedron("Cu", noshells=5)


#### 🟢インデックスでピンポイントに指定する

In [ ]:
# test_slabの0番目の原子を探す
find_atom_by_index(test_slab, 0)

#### 🟢指定した原子のインデックスを調べる

In [ ]:
# Cu原子のインデックスを調べる
find_indices_by_symbol(test_slab, "Cu")

#### 🟢隣接原子を探す

In [ ]:
# test_slabの0番目の原子の、隣接原子を探す
get_neighbors(test_slab, 0)